In [1]:
from Lab1_CausalGenerationResultAnalysis.M_Star import generate_BNN_MStar
from Lab1_CausalGenerationResultAnalysis.GenerateBNAndData import *
from Lab1_CausalGenerationResultAnalysis.evaluate import predict, calculate_metrics
import pandas as pd

In [2]:
aim_graph = 'Child'

if aim_graph == 'Asia':
    define_df_pth = "/Users/andrewlee/Desktop/Projects/实验室/114项目/Lab1_CausalGenerationResultAnalysis/Asia/asiaDefineChinese.csv"

    bif_file_path = '/Users/andrewlee/Desktop/Projects/实验室/114项目/Lab1_CausalGenerationResultAnalysis/Asia/asia.bif'

    aim = ['xray', 'dysp']

elif aim_graph == 'Insurance':
    define_df_pth = "/Users/andrewlee/Desktop/Projects/实验室/114项目/Lab1_CausalGenerationResultAnalysis/Insurance/InsuranceDefineChinese.csv"

    bif_file_path = '/Users/andrewlee/Desktop/Projects/实验室/114项目/Lab1_CausalGenerationResultAnalysis/Insurance/insurance.bif'

    aim = ['PropCost', 'ThisCarCost', 'ThisCarDam', 'OtherCarCost', 'ILiCost']

elif aim_graph == 'Child':
    define_df_pth = "/Users/andrewlee/Desktop/Projects/实验室/114项目/Lab1_CausalGenerationResultAnalysis/Child/ChildDefineChinese.csv"

    bif_file_path = '/Users/andrewlee/Desktop/Projects/实验室/114项目/Lab1_CausalGenerationResultAnalysis/Child/child.bif'

    aim = ['GruntingReport', 'LowerBodyO2', 'RUQO2', 'CO2Report', 'XrayReport']

else:
    raise Exception("aim_graph not found!")

# 读取节点信息

In [3]:
define_df = pd.read_csv(define_df_pth)
node_names = list(define_df['var_name'])
node_detail = list(define_df['var_description'])
node_detail

['婴儿出生时血液中氧气不足',
 '婴儿高铁血红蛋白血症',
 '婴儿出现疾病时的年龄',
 '左心室肥厚',
 '动脉导管中的血流',
 '氧合血和脱氧血的混合',
 '肺部血管的状态',
 '肺部血流量低',
 '存在疾病',
 '身体各部位均匀分布的低氧区域',
 '吸氧时的低氧状态',
 '体内二氧化碳水平',
 '进行了胸部X光检查',
 '婴儿发出的哼声',
 '有关左心室肥厚的报告',
 '下身的氧含量',
 '右大腿前侧肌肉的氧含量',
 '报告血液中高水平二氧化碳的文件',
 '肺部过度充血的X光报告',
 '有关婴儿发出哼声的报告']

# 读取原BN模型

In [4]:

origin_model = define_bayesian_network_from_bif(bif_file_path)
type(origin_model)

pgmpy.models.BayesianNetwork.BayesianNetwork

# 读取M*的BN模型

In [5]:
MStar_model = generate_BNN_MStar(node_names, node_detail)
MStar_model.edges()

waiting for LLM response of M*(stage 1/2):


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


waiting for LLM response of M*(stage 2/2)


INFO:httpx:HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


illegal network structure


TypeError: can only concatenate str (not "list") to str

# 反演生成测试数据

In [ ]:
samples_data = generate_samples(origin_model, 5000)
origin_model.remove_cpds()

# 评测

## 评测原模型

In [ ]:
predict_result = predict(origin_model, samples_data, aim)

In [ ]:
res_json = calculate_metrics(predict_result, samples_data, aim)

## 评测新模型

In [ ]:
samples_data

In [ ]:
predict_result = predict(MStar_model, samples_data, aim)

In [ ]:
res_json1 = calculate_metrics(predict_result, samples_data, aim)

## 汇总两个模型的效果

In [ ]:
# 画图比较res_json和res_json1中各aim中指标的Accuracy、Precision、Recall、F1的值
# res_json和res_json1是字典，格式为{'accuracy': accuracy, 'recall': recall, 'precision': precision, 'f1_score': f1}

import matplotlib.pyplot as plt

def plot_comparison(res_json, res_json1, metrics_lst):
    """
    绘图比较两个结果字典中各aim中指标的Accuracy、Precision、Recall、F1的值
    :param res_json: 第一个结果字典
    :param res_json1: 第二个结果字典
    :param metrics_lst: 需要比较的指标列表
    """
    aims = list(res_json.keys())
    width = 0.35  # 柱状图的宽度

    for aim in aims:
        plt.figure(figsize=(5, 3))
        x = metrics_lst  # x轴为指标名称
        y1 = [res_json[aim][metric] for metric in metrics_lst]
        y2 = [res_json1[aim][metric] for metric in metrics_lst]

        plt.bar(x, y1, width, label='res_json')
        plt.bar([p + width for p in range(len(x))], y2, width, label='res_json1')

        plt.xlabel('Metrics')
        plt.ylabel('Values')
        plt.title(f'Comparison of Metrics for {aim}')
        plt.xticks([p + width / 2 for p in range(len(x))], x)
        plt.legend()
        plt.show()

plot_comparison(res_json, res_json1, ['accuracy', 'precision', 'recall', 'f1_score'])


In [ ]:
def plot_comparison_f1(res_json, res_json1):
    """
    绘一个图比较两个结果字典中各aim中指标的F1的值
    :param res_json: 第一个结果字典
    :param res_json1: 第二个结果字典
    """
    aims = list(res_json.keys())
    width = 0.35  # 柱状图的宽度

    # 创建 x 轴标签
    x = aims
    x_indices = range(len(x))

    # 提取 F1 值
    y1 = [res_json[aim]['f1_score'] for aim in aims]
    y2 = [res_json1[aim]['f1_score'] for aim in aims]

    # 创建图形
    plt.figure(figsize=(10, 6))
    plt.bar(x_indices, y1, width, label='originalGraph')
    plt.bar([i + width for i in x_indices], y2, width, label='MStarGraph')

    # 设置图形属性
    plt.xlabel('Aims')
    plt.ylabel('F1 Score')
    plt.title('Comparison of F1 Scores for Different Aims')
    plt.xticks([i + width / 2 for i in x_indices], x)
    plt.legend()

    # 显示图形
    plt.show()

plot_comparison_f1(res_json, res_json1)